In [1]:
import requests as req 
from bs4 import BeautifulSoup, CData, Tag
import time
import tqdm
import pickle
import pandas as pd
import json

In [2]:
# pd.set_option('display.max_columns',None)
# pd.set_option('display.max_rows',None)

In [3]:
sites = { 
    'Фонтанка.ру' : ['https://www.fontanka.ru/fontanka.rss', ''],                  
    'LENTA.RU' : ['https://lenta.ru/rss', ''],                  
    'TASS' : ['https://tass.ru/rss/v2.xml', ''],
    'ВЕДОМОСТИ' : ['https://www.vedomosti.ru/rss/news', '']           
}

In [4]:
sites

{'Фонтанка.ру': ['https://www.fontanka.ru/fontanka.rss', ''],
 'LENTA.RU': ['https://lenta.ru/rss', ''],
 'TASS': ['https://tass.ru/rss/v2.xml', ''],
 'ВЕДОМОСТИ': ['https://www.vedomosti.ru/rss/news', '']}

In [5]:
def parsing():
    data = { "data" : [] }
        
    for site in range(len(sites)):
        
        url = list(sites.values())[site][0]
                
        resp = req.get(url)

        soup = BeautifulSoup(resp.content, features="xml")

        tag_item = soup.find_all('item')

        for item in tag_item:
            data['data'].append({
                "title": item.title.text,
                "pubDate": item.pubDate.text,
                "site": list(sites.keys())[site],
            })

    return pd.DataFrame(data['data'])

In [6]:
df = parsing()
df.sample(8)

,title,pubDate,site
361,Экс-прокурор Раменского осужден на 15 лет за к...,"Mon, 23 Jan 2023 21:19:10 +0300",ВЕДОМОСТИ
243,В Архангельской области намерены создать первы...,"Tue, 24 Jan 2023 13:33:21 +0300",TASS
375,В Железногорске опровергли информацию о вооруж...,"Mon, 23 Jan 2023 17:41:35 +0300",ВЕДОМОСТИ
340,Минобороны показало уничтожение украинского та...,"Tue, 24 Jan 2023 12:19:43 +0300",ВЕДОМОСТИ
148,В Геническе нашли заминированный крупный тайни...,"Tue, 24 Jan 2023 10:31:00 +0300",LENTA.RU
413,Россия понизила дипломатический уровень с Эсто...,"Mon, 23 Jan 2023 12:22:47 +0300",ВЕДОМОСТИ
297,"Кремль не располагает данными о том, что Киев ...","Tue, 24 Jan 2023 12:44:11 +0300",TASS
158,Тодоренко пожаловалась на трудности материнства,"Tue, 24 Jan 2023 10:11:00 +0300",LENTA.RU


In [7]:
topics = pd.DataFrame(['Библиотека', 'Россия', 'Мир', 'Экономика', 'Интернет и СМИ',
       'Спорт', 'Культура', 'Из жизни', 'Силовые структуры',
       'Наука и техника', 'Бывший СССР', 'nan', 'Дом', 'Сочи', 'ЧМ-2014',
       'Путешествия', 'Ценности', 'Легпром', 'Бизнес', 'МедНовости',
       'Оружие', '69-я параллель', 'Культпросвет ', 'Крым'])

In [8]:
model = None

with (open('../../lenta.ru/models/knb_ppl_clf.pkl', 'rb')) as openfile:
    model = pickle.load(openfile)

In [9]:
predicted_topics = model.predict(df['title'])

In [10]:
df_news = pd.concat([pd.Series(predicted_topics), df], ignore_index=True, axis=1)

df_news = df_news.rename(columns={0: "category", 1: "title", 2: "pub_date", 3: "site"})

df_news['parsed_date'] = None
df_news['day_of_week'] = None
for index, row in enumerate(df_news['pub_date']):
    parsed = time.strptime(row, "%a, %d %b %Y %H:%M:%S %z")
    df_news['parsed_date'][index] = time.strftime('%Y-%m-%d %H:%M:%S', parsed)
    df_news['day_of_week'][index] = time.strftime('%a', parsed)

df_news.sample(8)

,category,title,pub_date,site,parsed_date,day_of_week
40,Культура,Обнаружена омолаживающая сердце на 10 лет мутация,"Tue, 24 Jan 2023 13:33:42 +0300",LENTA.RU,2023-01-24 13:33:42,Tue
260,Интернет и СМИ,"Фигуристы Алиев, Игнатов и Даниелян примут уча...","Tue, 24 Jan 2023 13:23:16 +0300",TASS,2023-01-24 13:23:16,Tue
45,Экономика,Ученые раскрыли влияние коротких интенсивных т...,"Tue, 24 Jan 2023 13:28:11 +0300",LENTA.RU,2023-01-24 13:28:11,Tue
106,Наука и техника,Минобороны России показало бой «Терминатора» с...,"Tue, 24 Jan 2023 11:42:00 +0300",LENTA.RU,2023-01-24 11:42:00,Tue
413,Бывший СССР,Россия понизила дипломатический уровень с Эсто...,"Mon, 23 Jan 2023 12:22:47 +0300",ВЕДОМОСТИ,2023-01-23 12:22:47,Mon
170,Россия,ФСБ и бойцы СОБР Росгвардии задержали в Калини...,"Tue, 24 Jan 2023 09:50:00 +0300",LENTA.RU,2023-01-24 09:50:00,Tue
484,Культура,Маск назвал «безжалостной эскалацией» планы СШ...,"Sat, 21 Jan 2023 09:32:57 +0300",ВЕДОМОСТИ,2023-01-21 09:32:57,Sat
39,Россия,В Госдуме изучат вопрос о конфискации имуществ...,"Tue, 24 Jan 2023 13:34:00 +0300",LENTA.RU,2023-01-24 13:34:00,Tue


In [22]:
from kafka import KafkaProducer

producer = KafkaProducer(bootstrap_servers='localhost:9092')

def write_to_kafka(df):
    hm = {}
    for _, row in df.iterrows():
        hm["category"] = row["category"]
        hm["title"] = row["title"]
        hm["site"] = row["site"]
        hm["pub_date"] = row["parsed_date"]        
        hm["day_of_week"] = row["day_of_week"]
        producer.send('foobar', json.dumps(hm).encode('utf-8'))
        producer.flush()

write_to_kafka(df_news)

In [12]:
import happybase

connection = happybase.Connection('localhost')
table = connection.table('news')

def write_to_hbase(df):
    for i, row in df.iterrows():
        table.put(str.encode(str(i)), {
            b"cf:category": str.encode(row["category"]),
            b"cf:title": str.encode(row["title"]),
            b"cf:site": str.encode(row["site"]),
            b"cf:pub_date": str.encode(row["parsed_date"]),
            b"cf:day_of_week": str.encode(row["day_of_week"])
            })

write_to_hbase(df_news)